# Import Library 

In [1]:
import ephem
import pandas as pd 
import numpy as np 
from datetime import datetime
from datetime import timedelta
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

# Menentukan Latitude dan Longitude Lokasi menggunakan Modul Geopy

In [2]:
data = {'City':['Aceh', 'Padang', 'Palembang', 'Tangerang', 'Jakarta', 'Bandung']}
     
df = pd.DataFrame(data) 

In [3]:
longitude = []
latitude = []
    
def findGeocode(city):

    try:
          
        geolocator = Nominatim(user_agent="your_app_name")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)    
   
for i in (df["City"]):
      
    if findGeocode(i) != None:
        loc = findGeocode(i)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [4]:
df["Long"] = longitude
df["Lat"] = latitude
#df['Long'] = df['Long'].astype(str)
#df['Lat'] = df['Lat'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    6 non-null      object
 1   Long    6 non-null      object
 2   Lat     6 non-null      object
dtypes: object(3)
memory usage: 272.0+ bytes


# Prakira Hilal

In [26]:
def observer(lokasi, longitude, latitude, date):
    obs = ephem.Observer()
    obs.lat = latitude
    obs.lon = longitude
    obs.date = date #waktu pengamatan hilal: YYYY/MM/DD HH:MM:SS (waktu maghrib di lokasi dalam utc) 
    #obs.date = ephem.localtime(obs.date) #convert utc ke waktu lokal
    print("Pengamatan hilal dilakukan pada {} WIB".format(ephem.localtime(obs.date)))
    
    # Sun-Moon Elongation and its Altitude
    Moon = ephem.Moon()
    Moon.compute(obs)
    print("\nKetinggian bulan pada {} WIB sebesar {} derajat \n"
          "dan elongasi bulan sebesar {} derajat".format(ephem.localtime(obs.date),
                                                         Moon.alt, Moon.elong))
    
    # New Moon
    b = date.split(' ', 1)
    b = b[0]
    d = ephem.next_new_moon(b)
    d = ephem.to_timezone(d, ephem.UTC)
    #d = ephem.localtime(d)
    print("\nWaktu ijtimak terjadi pada: {} WIB".format(d))
    
    # Age of the Moon when does hilal observation
    d_obs = ephem.Date(obs.date)
    d_obs = ephem.to_timezone(d_obs, ephem.UTC)
    age = d_obs - d 
    print("Bulan pada saat pengamatan berumur {}".format(age))
        
    # MABIMS Criteria
    alt_mabims = ephem.degrees('2:00:00')
    elong_mabims = ephem.degrees('3:00:00')
    age_mabims = timedelta(hours=8)


    if Moon.alt >= alt_mabims:
        if Moon.elong >= elong_mabims:
            if age >= age_mabims:    
                print("\n###### Diperkirakan akan terlihat hilal di {} #######".format(lokasi))
            else:
                print("\n####### Diperkirakan hilal tidak akan terlihat hilal di {} #######".format(lokasi))
        else:
            print("\n####### Diperkirakan hilal tidak akan terlihat hilal di {} #######".format(lokasi))
    else:
        print("\n####### Diperkirakan hilal tidak akan terlihat hilal di {} #######".format(lokasi))
    
    
    return observer

In [27]:
#Data lokasi, longitude, dan Latitude
df

,City,Long,Lat
0,Aceh,97.0253024,4.3685491
1,Padang,100.3632561,-0.9247587
2,Palembang,104.756857,-2.9888297
3,Tangerang,106.6384468,-6.1760311
4,Jakarta,106.827183,-6.1753942
5,Bandung,107.6049539,-6.9344694


In [29]:
# Prakiraan di kota Tangerang
# Observer(lokasi, longitude, latitude, tanggal-waktu YYYY-MM-DD HH:MM:SS)
observer('kota Tangerang', '106.6384468', '-6.1760311', '2021/04/12 10:58:00')

Pengamatan hilal dilakukan pada 2021-04-12 17:58:00 WIB

Ketinggian bulan pada 2021-04-12 17:58:00 WIB sebesar 2:35:03.0 derajat 
dan elongasi bulan sebesar 5:18:08.1 derajat

Waktu ijtimak terjadi pada: 2021-04-12 02:30:47.671991+00:00 WIB
Bulan pada saat pengamatan berumur 8:27:12.328009

###### Diperkirakan akan terlihat hilal di kota Tangerang #######


<function __main__.observer(lokasi, longitude, latitude, date)>

## catatan:
- Altitude/Ketinggian bulan bernilai __minus__ menunjukkan bahwa bulan berada di bawah horizon
- Umur Bulan bernilai __minus__ menunjukkan bahwa bulan belum mencapai ijtimak 